<H1> LLM Tutor Chat</H1><br>
<H3> User should be able to ask questions and the tool queries LLM and returns the results </h3>

In [16]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import json

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [4]:
# For Ollama, you can use the following constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL_LLAMA = "llama3.2"
MODEL_GPT= "gpt-4o-mini"

In [5]:
# Here is the question to be answered
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [6]:
# Querying GPT to answer with streaming

def get_answer(question: str, model: str = MODEL_GPT):
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role":"system","content": "You are a helpful assistant that answers questions"},
            {"role":"user","content":question}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))


In [21]:
def get_answer(question: str, model: str = MODEL_GPT):
    if "llama" in model:
        print("Using Ollama for model:", model)
        url = OLLAMA_API
        data = {
            "model": model,
            "messages": [
                {"role": "system", "content": "You are a helpful assistant that answers questions"},
                {"role": "user", "content": question}
            ]
        }
        response = requests.post(url, headers=HEADERS, json=data)
        try:
            result = response.json()['message']['content']
        except json.decoder.JSONDecodeError:
            # Try to parse the first JSON object if multiple are present
            first_json = response.text.split('\n')[0]
            result = json.loads(first_json)['message']['content']
        display(Markdown(result))
    else:
        print("Using OpenAI for model:", model)
        response = openai.chat.completions.create(
            model=model,
            messages=[
                {"role":"system","content": "You are a helpful assistant that answers questions"},
                {"role":"user","content":question}
            ],
        )
        result = response.choices[0].message.content
        display(Markdown(result))

In [23]:
get_answer(question, model=MODEL_GPT)

Using OpenAI for model: gpt-4o-mini


The line of code you provided is a Python statement that utilizes the `yield from` syntax in conjunction with a set comprehension. Let's break it down step by step to understand its functionality and purpose.

1. **Set Comprehension**:
   - The expression `{book.get("author") for book in books if book.get("author")}` is a set comprehension. 
   - It iterates over a collection `books`, which is expected to be a list (or any iterable) of dictionaries (or similar objects).
   - For each `book` in `books`, it attempts to retrieve the value associated with the key `"author"` using the `get` method. This method returns `None` if the key does not exist.
   - The `if book.get("author")` part is a condition that filters the authors: it ensures that only books with a non-`None` author are included in the resulting set.
   - The result is a set of unique author names (or `None` values if they were present) from the `books` collection.

2. **Yielding Results**:
   - The `yield from` statement is used within a generator function. It allows the function to yield all values from an iterable (in this case, the set created by the set comprehension) one by one.
   - This means that if this line is part of a generator function, calling that function would result in yielding each unique author's name to the calling context until there are no more authors left to yield.

### Summary
In summary, this line of code:
- Creates a set of unique author names from a list of books, filtering out any books that do not have an author.
- If part of a generator function, it allows the function to yield each unique author name one at a time, facilitating iteration over those names.

### Example
For example, if `books` were defined as:
```python
books = [
    {"title": "Book A", "author": "Author 1"},
    {"title": "Book B", "author": "Author 2"},
    {"title": "Book C"},  # No author
    {"title": "Book D", "author": "Author 1"},  # Duplicate author
]
```
The resulting set comprehension would yield:
```python
{"Author 1", "Author 2"}
```
And if this was part of a generator function, it would allow you to loop through these authors one by one.

<H1> Getting the answer using stream </H1>
<b>change this so that the results stream back from OpenAI, with the familiar typewriter animation</b>

In [10]:
def get_answer_stream(question: str, model: str = MODEL_GPT):
    stream = openai.chat.completions.create(
        model=model,
        messages=[
            {"role":"system","content": "You are a helpful assistant that answers questions"},
            {"role":"user","content":question}
        ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""),display_id=True)
    for chunk in stream:
       response += chunk.choices[0].delta.content or ''
       response = response.replace("```","").replace("markdown","")
       update_display(Markdown(response),display_id=display_handle.display_id)

In [11]:
get_answer_stream(question, model=MODEL_GPT)

The line of code you've provided uses Python's generator feature along with a set comprehension. Let's break it down step by step:

1. **Set Comprehension**: The part inside the curly braces `{ ... for book in books if book.get("author") }` is a set comprehension. This creates a set of unique values based on the criteria defined within the comprehension:
   - `book` is iterating over a collection called `books`.
   - `book.get("author")` retrieves the value associated with the key `"author"` from each `book` dictionary.
   - The `if book.get("author")` condition ensures that only books that have a non-null (truthy) author value are included in the set.

   So the set comprehension collects the authors of the books, but only those books that have a valid `"author"` entry.

2. **Yield from**: The `yield from` keyword is used in Python to yield all values from an iterable. In this case, it yields each author from the set created by the comprehension. By using `yield from`, it allows the function (which this line is likely a part of) to yield each item one at a time, rather than returning the entire set at once.

3. **Purpose**: The overall effect of this line is to create a generator that yields each unique author from the `books` collection, skipping any books that do not have an author listed. The use of `yield from` efficiently produces each value without loading all the values into memory at once, which is beneficial when dealing with larger datasets.

### Example

For instance, if you have a list of books structured like this:

python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": None},
    {"title": "Book 4", "author": "Author A"},
]


The set comprehension would create the set `{"Author A", "Author B"}` (note that `Author A` only appears once due to the uniqueness property of sets). The resulting code would yield `Author A` and then `Author B` one after the other when the generator is iterated over. 

### Summary

In conclusion, this line of code efficiently extracts and yields unique authors from a list of book dictionaries, filtering out entries without a valid author value.